In CoreML Neural Network Specification version 4 (which is available from iOS 13 and MacOS 10.15), several "control-flow" layers have been added. CoreML spec is described in the protobuf format and for a list of all supported layer types and documentation, see [here](https://github.com/apple/coremltools/blob/master/mlmodel/format/NeuralNetwork.proto).

In this notebook, we build a neural network that uses a few of the new control flow layers. We will write a simple python program to compute the largest eigenvalue of a given matrix and then show how a neural network can be built to replicate that program in an mlmodel.

We choose the [power iteration method](https://en.wikipedia.org/wiki/Power_iteration). It is a simple iterative algorithm. Given a square matrix, $A$ of dimensions $n\times n$, it computes the largest eigenvalue (by magnitude) and the corresponding eigenvector (the algorithm can be adapted to compute all the eigenvalues, however we do not implement that here). 

Here is how the algorithm works. Pick a normalized random vector to start with, $x$, of dimension $n$. Repetitively, multiply it by the matrix and normalize it, i.e., $x\leftarrow Ax$ and $x\leftarrow \frac{x}{\left \| x \right \|}$. Gradually the vector converges to the largest eigenvector. Simple as that! 
There are a few conditions that the matrix should satisfy for this to happen, but let us not worry about it for this example. 
For now we will assume that the matrix is real and symmetric, this guarantees the eigenvalues to be real. 
After we have the normalized eigenvector, the corresponding eigenvalue can be computed by the formula $x^TAx$



Let's code this up in Python using Numpy!

In [1]:
import numpy as np
import copy

np.random.seed(8) # try different seeds to play with the number of iterations it takes for convergence!

'''
Use power method to compute the largest eigenvalue of a real symmetric matrix
'''

convergence_tolerance = 1e-6 # decrease/increase to trade off precision
number_of_iterations = 100 # decrease/increase to trade off precision

def power_iteration(matrix, starting_vector):
    x = copy.deepcopy(starting_vector)
    for i in range(number_of_iterations):
        y = np.matmul(A,x)
        #normalize
        y = y / np.sqrt(np.sum(y**2))
        # compute the diff to check for convergence
        # we use cosine difference as both vectors are normalized and can get
        # rotated by 180 degrees between iterations
        diff = 1-abs(np.dot(x,y))
        # update x
        x = y
        print('{}: diff: {}'.format(i, diff))
        if diff < convergence_tolerance: 
            break

    x_t = np.transpose(x)
    eigen_value = np.matmul(x_t, np.matmul(A,x))
    return eigen_value, x
    

# define the symmetric real matrix for which we need the eigenvalue. 
A = np.array([[4,-5], [-5,3]], dtype=np.float)

# a random starting vector
starting_vector = np.random.rand(2)
starting_vector = starting_vector / np.sqrt(np.sum(starting_vector**2)) ## normalize it
 
eigen_value, eigen_vector = power_iteration(A, starting_vector)

print('Largest eigenvalue: %.4f ' % eigen_value)
print('Corresponding eigenvector: ', eigen_vector)

0: diff: 6.69187030143e-05
1: diff: 0.00208718410489
2: diff: 0.0614522880272
3: diff: 0.771617699317
4: diff: 0.193129218664
5: diff: 0.0075077446807
6: diff: 0.000241962094403
7: diff: 7.74407193072e-06
8: diff: 2.47796068775e-07
Largest eigenvalue: 8.5249 
('Corresponding eigenvector: ', array([-0.74152421,  0.67092611]))


We see that in this case, the algorithm converged, given our specified toelrance, in 9 iterations. 
To confirm whether the eigenvalue is correct, lets use the "linalg" sub-package of numpy. 

In [2]:
from numpy import linalg as LA

e, v = LA.eig(A)
idx = np.argmax(abs(e))
print('numpy linalg: largest eigenvalue: %.4f ' % e[idx])
print('numpy linalg: first eigenvector: ', v[:,idx])

numpy linalg: largest eigenvalue: 8.5249 
('numpy linalg: first eigenvector: ', array([ 0.74145253, -0.67100532]))


Indeed we see that the eigenvalue matches with our power iteration code. The eigenvector is rotated by 180 degrees, but that is fine.

Now, lets build an mlmodel to do the same. We use the builder API provided by coremltools to write out the protobuf messages. 

In [3]:
import coremltools
import coremltools.models.datatypes as datatypes
from coremltools.models.neural_network import NeuralNetworkBuilder

input_features = [('matrix', datatypes.Array(*(2,2))),
                  ('starting_vector', datatypes.Array(*(2,)))]

output_features = [('maximum_eigen_value', datatypes.Array(*(1,))), 
                   ('eigen_vector', None),
                   ('iteration_count', datatypes.Array(*(1,)))]

builder = NeuralNetworkBuilder(input_features, output_features, disable_rank5_shape_mapping=True)

# convert the starting_vector which has shape (2,) to shape (2,1) 
# so that it can be used by the Batched-MatMul layer
builder.add_expand_dims('expand_dims', 'starting_vector', 'x', axes=[-1])
builder.add_load_constant_nd('iteration_count', 'iteration_count',
                             constant_value=np.zeros((1,)),
                             shape=(1,))

# start building the loop
loop_layer = builder.add_loop('loop', max_iterations=number_of_iterations)
# get the builder object for the "body" of the loop
loop_body_builder = NeuralNetworkBuilder(nn_spec=loop_layer.loop.bodyNetwork)

# matrix multiply
# input shapes: (n,n),(n,1)
# output shape: (n,1)
loop_body_builder.add_batched_mat_mul('bmm.1', input_names=['matrix','x'], output_name='y')
# normalize the vector
loop_body_builder.add_reduce_l2('reduce', input_name='y', output_name='norm', axes = 0)
loop_body_builder.add_divide_broadcastable('divide', ['y','norm'], 'y_normalized')

# find difference with previous, which is computed as (1 - abs(cosine diff))
loop_body_builder.add_batched_mat_mul('cosine', ['y_normalized', 'x'], 'cosine_diff', transpose_a=True)
loop_body_builder.add_unary('abs_cosine','cosine_diff','abs_cosine_diff', mode='abs')
loop_body_builder.add_activation('diff', non_linearity='LINEAR',
                                 input_name='abs_cosine_diff',
                                 output_name='diff', params=[-1,1])

# update iteration count
loop_body_builder.add_activation('iteration_count_add', non_linearity='LINEAR',
                                 input_name='iteration_count',
                                 output_name='iteration_count_plus_1', params=[1,1])
loop_body_builder.add_copy('iteration_count_update', 'iteration_count_plus_1', 'iteration_count')

# update 'x'
loop_body_builder.add_copy('update_x', 'y_normalized', 'x')

# add condition to break from the loop, if convergence criterion is met
loop_body_builder.add_less_than('cond', ['diff'], 'cond', alpha=convergence_tolerance)
branch_layer = loop_body_builder.add_branch('branch_layer', 'cond')
builder_ifbranch = NeuralNetworkBuilder(nn_spec=branch_layer.branch.ifBranch)
builder_ifbranch.add_loop_break('break')

# now we are out of the loop, compute the eigenvalue
builder.add_batched_mat_mul('bmm.2', input_names=['matrix','x'], output_name='x_right')
builder.add_batched_mat_mul('bmm.3', input_names=['x','x_right'], output_name='maximum_eigen_value', transpose_a=True)
builder.add_squeeze('squeeze', 'x', 'eigen_vector', squeeze_all=True)

spec = builder.spec
model = coremltools.models.MLModel(spec)

Okay, so now we have the mlmodel spec. Before we call predict on it, lets print it out to check whether everything looks okay. We use the utility called "print_network_spec"

In [4]:
from  coremltools.models.neural_network.printer import print_network_spec
print_network_spec(spec, style='coding')

Inputs:
  matrix [2, 2]
  starting_vector [2]
Outputs:
  maximum_eigen_value [1]
  eigen_vector []
  iteration_count [1]


def model(matrix, starting_vector) :
	x =  expandDims (starting_vector)
	iteration_count =  loadConstantND (shape = (1,),  value = [0.0] )
 	loop ()
		y =  batchedMatmul (matrix, x)
		norm =  reduceL2 (y)
		y_normalized =  divideBroadcastable (y, norm)
		cosine_diff =  batchedMatmul (y_normalized, x)
		abs_cosine_diff =  unary (cosine_diff)
		diff =  activation (abs_cosine_diff)
		iteration_count_plus_1 =  activation (iteration_count)
		iteration_count =  copy (iteration_count_plus_1)
		x =  copy (y_normalized)
		cond =  lessThan (diff)
 		branch (cond)
 		IfBranch:
 			loopBreak
	x_right =  batchedMatmul (matrix, x)
	maximum_eigen_value =  batchedMatmul (x, x_right)
	eigen_vector =  squeeze (x)
 
	return maximum_eigen_value, eigen_vector, iteration_count


In [5]:
# call predict on CoreML model
input_dict = {}
input_dict['starting_vector'] = starting_vector
input_dict['matrix'] = A.astype(np.float)

output = model.predict(input_dict)
coreml_eigen_value = output['maximum_eigen_value']
coreml_eigen_vector = output['eigen_vector']

print('CoreML computed eigenvalue: %.4f' % coreml_eigen_value)
print('CoreML computed eigenvector: ', coreml_eigen_vector, coreml_eigen_vector.shape)
print('CoreML iteration count: %d' % output['iteration_count'])

CoreML computed eigenvalue: 8.5249
('CoreML computed eigenvector: ', array([-0.74152416,  0.67092603]), (2,))
CoreML iteration count: 9


Indeed the output matches with our python program. 

Although, we do not do it here, the parameters "convergence_tolerance" and "number_of_iterations" can be made as network inputs, so that their value can be modifed at runtime. 

Currently, the input shapes to the Core ML model are fixed, $(2, 2)$ for the matrix and $(2,)$ for the starting vector. However, we can add shape flexibility so that the same mlmodel can be run on different input sizes. There are two ways to specify shape flexibility, either through "ranges" or via a list of "enumerated" shapes. Here we specify the latter.

In [6]:
from coremltools.models.neural_network import flexible_shape_utils

# (2,2) has already been provided as the default shape for "matrix" 
# during initialization of the builder,
# here we add two more shapes that will be allowed at runtime
flexible_shape_utils.add_multiarray_ndshape_enumeration(spec, 
                                                        feature_name='matrix',
                                                        enumerated_shapes=[(3,3), (4,4)])

# (2,) has already been provided as the default shape for "matrix" 
# during initialization of the builder,
# here we add two more shapes that will be allowed at runtime
flexible_shape_utils.add_multiarray_ndshape_enumeration(spec, 
                                                        feature_name='starting_vector',
                                                        enumerated_shapes=[(3,), (4,)])

model = coremltools.models.MLModel(spec)

In [7]:
# lets run the model with a (3,3) matrix 
A = np.array([[1, -6, 8], [-6, 1, 5], [8, 5, 1]], dtype=np.float)

starting_vector = np.random.rand(3)
starting_vector = starting_vector / np.sqrt(np.sum(starting_vector**2)) ## normalize it

eigen_value, eigen_vector = power_iteration(A, starting_vector)

print('python code: largest eigenvalue: %.4f ' % eigen_value)
print('python code: corresponding eigenvector: ', eigen_vector)

0: diff: 0.99757552989
1: diff: 0.718149467089
2: diff: 0.492558374678
3: diff: 0.325410135011
4: diff: 0.208606358183
5: diff: 0.130795340624
6: diff: 0.0807677916817
7: diff: 0.0493798553633
8: diff: 0.0299993308647
9: diff: 0.0181536364413
10: diff: 0.0109588786353
11: diff: 0.00660585926588
12: diff: 0.0039783687005
13: diff: 0.00239467498795
14: diff: 0.00144094325621
15: diff: 0.000866886171118
16: diff: 0.000521466038849
17: diff: 0.00031366000502
18: diff: 0.000188657339187
19: diff: 0.000113468967192
20: diff: 6.82454629412e-05
21: diff: 4.1045582895e-05
22: diff: 2.46863363353e-05
23: diff: 1.48472285797e-05
24: diff: 8.92962598664e-06
25: diff: 5.37057288463e-06
26: diff: 3.23003808245e-06
27: diff: 1.94264962894e-06
28: diff: 1.16837216313e-06
29: diff: 7.02696602684e-07
python code: largest eigenvalue: -11.7530 
('python code: corresponding eigenvector: ', array([ 0.61622756,  0.52125649, -0.59038569]))


In [8]:
from numpy import linalg as LA

e, v = LA.eig(A)
idx = np.argmax(abs(e))
print('numpy linalg: largest eigenvalue: %.4f ' % e[idx])
print('numpy linalg: first eigenvector: ', v[:,idx])

numpy linalg: largest eigenvalue: -11.7530 
('numpy linalg: first eigenvector: ', array([-0.61583909, -0.5213392 ,  0.59071791]))


In [9]:
input_dict['starting_vector'] = starting_vector
input_dict['matrix'] = A.astype(np.float)

output = model.predict(input_dict)
coreml_eigen_value = output['maximum_eigen_value']
coreml_eigen_vector = output['eigen_vector']

print('CoreML computed eigenvalue: %.4f' % coreml_eigen_value)
print('CoreML computed eigenvector: ', coreml_eigen_vector, coreml_eigen_vector.shape)
print('CoreML iteration count: %d' % output['iteration_count'])

CoreML computed eigenvalue: -11.7530
('CoreML computed eigenvector: ', array([ 0.61622757,  0.52125645, -0.59038568]), (3,))
CoreML iteration count: 30
